#Kill-Neighbors:

Logic: I've been operating on a particular guess, which I believe I've proven by now, that there are a number of "local minima". That is: a. Each time you run the program, it spits out a number (in this case, 100) of sequences which are not random, but are clusters of related sequences, whose binding affinities are also related to eachother. b. On sequential runs of the program, the groups produced "overlap", around the aforementioned clusters. -- Taken together, it is implied then, that there are one or more sequences in a cluster that will have the highest API-score in that cluster. Running the program more times, and gathering more sequences, maes it more-and-more likely that you'll hit that "local-minimum". So, generating more-and-more sequences, while killing off the sub-optimaal sequences, should function as a sort of optimization problem. "How close do you want to get?"/"How likely do you want to be to get the optimal solution?" -> That depends on how many times you can afford to run Apta-MCTS, and the neighbor-killing function, which depends on your computing power.

Idelly, I'll be able to run Apta-MCTS directly in colab in the future, taking advantage of Google's runtime, and making it much simpler to iterate.

kill_neighbors(D=0.1, N=0)
- The function takes in a list of sequences, and their respective API scores.
- Function determines the number of neighbors a sequence has, based on 'D', the neighbourhood cutoff distance.
- The neighborhoods are then ranked by size.
- For the neighborhoods of the largest size, the sequence in that neighborhood with the lowest API score is "killed".

-Iterate the whole process until the largest neighborhood is of the maximum size, N.


## Import the data:

In [ ]:
#From StackExchange
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving Exp_1_1.csv to Exp_1_1.csv
Saving Exp_1_16.csv to Exp_1_16.csv
Saving Exp_2_1.csv to Exp_2_1.csv
Saving Exp_2_16.csv to Exp_2_16.csv
Saving Exp_3_1.csv to Exp_3_1.csv
Saving Exp_3_16.csv to Exp_3_16.csv
Saving Exp_4_1.csv to Exp_4_1.csv
Saving Exp_4_16.csv to Exp_4_16.csv
Saving Exp_5_1.csv to Exp_5_1.csv
Saving Exp_5_16.csv to Exp_5_16.csv


# Function Definitions:

## Installing packages and such:

In [ ]:
pip install tn93 --q

In [ ]:
pip install Bio --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.6 MB/s eta 0:00:00


In [ ]:
import tn93

In [ ]:
import pandas as pd

In [ ]:
from tn93 import tn93 as model

In [ ]:
from Bio import SeqIO

## 'make_necessary_objects' function

In [ ]:
def make_necessary_objects(file_list):
  frames = []
  for file in file_list:
    name = file + 'df'
    name = pd.read_csv(file, delimiter=',', encoding_errors = 'replace')
    name['file'] = file
    frames.append(name)
  fin_df = pd.concat(frames)
  #Make the 'API_dict'
  o_seqs = fin_df.drop(['secondary_structure', 'minimum_free_energy', 'file'],axis=1)
  o_seqs = o_seqs.values
  API_dict = {}
  for n in o_seqs:
    API_dict[n[1]] = n[0]
  #Make the 'E_dict'
  e_seqs = fin_df.drop(['aptamer_protein_interaction_score','secondary_structure', 'file'],axis=1)
  e_seqs = e_seqs.values
  e_seqs
  E_dict = {}
  for n in e_seqs:
    E_dict[n[0]] = n[1]
  #Make 'pairwise_tn_df'
  pairwise_tn = []
  for n in API_dict:
    row = []
    for m in API_dict:
      try: #A few of the pairs return a 'math domain error'
        row.append(float(tamura_nei(n,m)[2]))
      except:
        #print('Math_error', n,m)
        row.append(1.0)    #Impute the maximum distance.
    pairwise_tn.append(row)
  pairwise_tn_df = pd.DataFrame(pairwise_tn, columns=API_dict, index=API_dict)

  #Return everything:
  return(fin_df, API_dict, E_dict, pairwise_tn_df)

## Tamura_Nei Distance function:

In [ ]:
def tamura_nei(s1,s2):
  #Make the SeqRecord objects for each.
  seq1 = SeqIO.SeqRecord(seq=s1)
  seq1.id=s1
  seq2 = SeqIO.SeqRecord(seq=s2)
  seq2.id=s2
  #And feed into the model.
  tn_model = model.TN93(minimum_overlap=8)
  distance = tn_model.tn93_distance(seq1, seq2, "RESOLVE")
  #The output looks like a list, but it isn't. Annoying.
  distance = distance.split(',')
  return(distance)

## get_neighbors: Function.

In [ ]:
def get_neighbors(pairwise_tn_df, D=0.1):
  groups = {} #Since each group is now a tuple, encoding the whole thing as a dictionary may prove useful.

  for index, row in pairwise_tn_df.iterrows():
      centroid = (f"{index}")
      neighbors = []
      for column, value in row.items():
          if float(f"{value}") <= D and float(f"{value}") != 0: #Add self-match condition.
            neighbors.append(f"{column}")

      #Add to the dictionary:
      groups[centroid] = neighbors

  return(groups)

## Hit-list:

1. 'NS': The first algorithm that I came up with was: a. Look at the sizes of all neighborhoods, and determine the maximim size. b. For all of that size, determine which sequence has the lowest API-score. c. Add that sequence to the hit-list, to be pruned. -- Note that sequences are prioritized for pruning by proximity to a sequence that has a lot of neighbors, by "neighborhood size".

2. 'API': Another potential algorithm: a. Look for all sequences which have the lowest API score in their own neighborhood. b. Of these, determine which has the largest neighborhood (is the most redundant), and add it to the hit-list.

In both cases, the function needs to return a "stop"  or something like it, if the largest group size is at or below the threshold 'N'. This would be how we iterate and stop, progressively weeding out sequences until we reach the desired size. Unless... maybe doing that is pointless.

### 'NS' Function

In [ ]:
def NS(inp):
  hit_list = []
  #This block simply gives the maximum size of any neighborhood.
  ls = []
  for n in inp:
    ls.append(len(inp[n]))
  ml = max(ls)
  #Iterate this block based on whether the loop has found something to prune yet:
  for m in range(0, ml):
    if len(hit_list) == 0:
      #Looking at all neighborhoods of this maximum size.
      for n in inp:
        if len(inp[n]) == (ml-m):  #Notice the new '-m' condition.
          #Create an output list.
          lw_APIs = []
          #Append the centroid
          lw_APIs.append([n, API_dict[n]])
          for i in inp[n]:
            #Append the neighbors
            lw_APIs.append([i,API_dict[i]])

          #To generate the spread of API scores in each list.
          lAPIs = []
          for j in lw_APIs:
            lAPIs.append(j[1])

          #Add all the prunable sequences to the hit-list:
          for j in lw_APIs:
            if j[1] == min(lAPIs) and j[1] != max(lAPIs):
              hit_list.append(j)

  return ('hit_list:', hit_list)

### API Function:

In [ ]:
def API(inp):
  hit_list = []
  provis_hit_list = []

  #First, to identify all sequences which are of the minimum API score in their own neighborhoood.
  for n in inp:
    #Create an output list.
    lw_APIs = []
    #Append the centroid
    lw_APIs.append([n, API_dict[n]])
    for i in inp[n]:
      #Append the neighbors
      lw_APIs.append([i,API_dict[i]])

    #To generate the spread of API scores in each list.
    lAPIs = []
    for j in lw_APIs:
      lAPIs.append(j[1])

    #Select those centroids which are a local minimum
    if API_dict[n] == min(lAPIs) and API_dict[n] != max(lAPIs):
      provis_hit_list.append([n, API_dict[n], len(inp[n])])

  #Selection of sequences from the provisional hit-list which have the lowest score.
  seq_APIs = []
  for seq in provis_hit_list:
    seq_APIs.append(seq[1])

  for seq in provis_hit_list:
    if seq[1] == min(seq_APIs):
      hit_list.append(seq)

  return(hit_list)

### API_E Function:

Added in this version of the program. If there are only impasse solutions at the level of the largest group, then we begin prunning by Free Energy Change.

In [ ]:
def API_E(inp):
  hit_list = []
  provis_hit_list = []

  #First, to identify all sequences which are of the minimum API score in their own neighborhoood.
  for n in inp:
    #Create an output list.
    lw_APIs = []
    #Append the centroid
    lw_APIs.append([n, API_dict[n],E_dict[n]])
    for i in inp[n]:
      #Append the neighbors
      lw_APIs.append([i,API_dict[i],E_dict[i]])

    #To generate the spread of API and E scores in each list.
    lAPIs = []
    lEs = []
    for j in lw_APIs:
      lAPIs.append(j[1])
      lEs.append(j[2])

    #Select those centroids which are a local minimum
    if API_dict[n] == min(lAPIs) and API_dict[n] != max(lAPIs):
      provis_hit_list.append([n, API_dict[n], len(inp[n])])


    #From the impasse solutions (where all neighbors have the same API_score, and so min(lAPIs) == max(lAPIs)),     # This section generates the differing behavior from 'API'
    #take the solution with the least free energy change (max value), and add it to the hit-list.
    elif API_dict[n] == min(lAPIs) and API_dict[n] == max(lAPIs):
      if E_dict[n] == max(lEs) and E_dict[n] != min(lEs):
        provis_hit_list.append([n, API_dict[n], len(inp[n])])

  #Selection of sequences from the provisional hit-list which have the lowest API score.
  seq_APIs = []
  for seq in provis_hit_list:
    seq_APIs.append(seq[1])

  for seq in provis_hit_list:
    if seq[1] == min(seq_APIs):
      hit_list.append(seq)

  return(hit_list)

### Combined Function:

In [ ]:
def hit_list(pairwise_tn_df, API_dict, D=0.1, alg='API_E'):
  inp = get_neighbors(pairwise_tn_df, D)
  if alg == 'NS':
    hl = NS(inp)
  elif alg == 'API':
    hl = API(inp)
  elif alg == 'API_E':
    hl = API_E(inp)
  elif alg != 'NS' and alg != 'API' and alg != 'API_E':
    print("Unaccepatable algorithm choice. Pick either: a. 'NS', b. 'API' or 'API_E'")

  hl_out = []
  for i in hl:
    hl_out.append(i[0])

  return(hl_out)

## Prune_Matrix

Remove all sequences on the hitlist from 'pairwise_tn_df.

In [ ]:
#Note that this and the pruning function are meant to recur:
#So to begin, input_df = pairwise_tn_df, but the next time, it will be the output of the prev. step
def prune_matrix(input_df, API_dict, D=0.1, alg='API'):
  pruned_df = input_df.copy()
  for seq in hit_list(input_df, API_dict, D, alg):
    pruned_df.drop([seq], axis=1, inplace=True)
    pruned_df.drop([seq], axis=0, inplace=True)
  return(pruned_df)

##. Kill_Neighbors:

Finally, the assembly:

1. Take in 'API_dict' and 'pairwise_tn_df'.

2. ['get_neighbors'->'build_hitlist'-> 'prune_matrix'] x Iterate until a cutoff is reached.

In [ ]:
def kill_neighbors(pairwise_tn_df, API_dict, E_dict, D=0.1, alg='API', lim=100, show=''):
  input_df = pairwise_tn_df.copy()

  count = 0
  output_dim = [(0, 0), input_df.values.shape]
  while count < lim and output_dim[count+1] != output_dim[count]:
    count +=1

    #A quick way to perhaps make the step-wise output available.   #Editted to make the names more intuitive.
    neighborhood_list = get_neighbors(input_df, D)
    if show == 'get_neighbors':
      print(neighborhood_list)
    kill_list = hit_list(input_df, API_dict, D, alg)
    if show == 'hit_list':
      print(kill_list)
    output_df = prune_matrix(input_df, API_dict, D, alg)
    if show == 'pairwise_tn_df':
      print(output_df)

    #The aim of doing this is that you should be able to see "convergence."
    #If the matrix from two successive iterations is the same size, the
    output_dim.append(output_df.values.shape)
    if show == 'dims':
      print(output_dim[count+1])
    input_df = output_df.copy()

  #Outputs the final distance matrix:
  if show == 'dist':
    return(output_df)

  else:
    survivors = []
    for i in output_df.columns:
      survivors.append([i, API_dict[i], E_dict[i]])
      survivors_df = pd.DataFrame(survivors, columns=['sequence','API_score', 'Free_Energy_Change'])
      survivors_df
    return(survivors_df)

# Usage:

Note: "Object_making" is generally the most time-consuming part of the process of using this program, hence why it's separate from the rest of the "neighbor killing" function. It allows the user to modify other arguments, apart from the input data, without having to remake the pairwise distance matrix (The matrix being of size n x n, where n = number of input sequences.)

In [ ]:
All_Sequences, API_dict, E_dict, pairwise_tn_df = make_necessary_objects(file_list=['Exp_1_1.csv','Exp_2_1.csv', 'Exp_3_1.csv', 'Exp_4_1.csv', 'Exp_5_1.csv'])

In [ ]:
#The only one of these that the user really might need to see.....
All_Sequences

,aptamer_protein_interaction_score,primary_sequence,secondary_structure,minimum_free_energy,file
0,0.400000,AACCGGTTUAAAUAAUUUACGUUTTGGCCAA,....((((.((((.......)))).))))..,-3.4,Exp_1_1.csv
1,0.400000,AACCGGTTAAUUAAUUUACGUGGTTGGCCAA,....(((((((((.......)))))))))..,-6.9,Exp_1_1.csv
2,0.400000,AACCGGTTAAUAAUUUACGCUAATTGGCCAA,....(((((((...........)))))))..,-4.7,Exp_1_1.csv
3,0.400000,AACCGGTTCAAAUAAUUUACGUUTTGGCCAA,....(((.((((..........)))))))..,-3.6,Exp_1_1.csv
4,0.400000,AACCGGTTCUGUAAUUUACGUGGTTGGCCAA,(((((((..........)).)))))......,-2.9,Exp_1_1.csv
...,...,...,...,...,...
95,0.371429,AACCGGTTCUAUAUGUGGUUGAGTTGGCCAA,..(((((((...........)))))))....,-3.5,Exp_5_1.csv
96,0.371429,AACCGGTTAGUGUGGUUGUUAGCTTGGCCAA,....((((((.((........))))))))..,-5.6,Exp_5_1.csv
97,0.371429,AACCGGTTAGUUUAAUGGUUGCUTTGGCCAA,....((((((.............))))))..,-3.6,Exp_5_1.csv
98,0.371429,AACCGGTTCUGGUGGGUAAUUGUTTGGCCAA,.((((....))))((.(((....))).))..,-6.1,Exp_5_1.csv


In [ ]:
test = kill_neighbors(pairwise_tn_df, API_dict, E_dict, D=0.4, alg='API', lim=100, show='')
test

,sequence,API_score,Free_Energy_Change
0,AACCGGTTAAGUUGUGGUUUAUGTTGGCCAA,0.485714,-4.8
1,AACCGGTTAAUUGUGGUUUAUGUTTGGCCAA,0.485714,-4.7
2,AACCGGTTGUGUGGUUUAUGAGCTTGGCCAA,0.485714,-3.6
3,AACCGGTTACGUUGUGGUUUAUGTTGGCCAA,0.485714,-5.4
4,AACCGGTTGUUGUGGUUUAUGUCTTGGCCAA,0.485714,-4.1
5,AACCGGTTGCGUUGUGGUUUAUGTTGGCCAA,0.485714,-5.0
6,AACCGGTTCGUUGUGGUUUAUGUTTGGCCAA,0.485714,-3.6
7,AACCGGTTAUGUUGUGGUUUAUGTTGGCCAA,0.485714,-4.2
8,AACCGGTTAUGUGGUUUAUGUUGTTGGCCAA,0.485714,-3.2
9,AACCGGTTAGUUGUGGUUUAUGGTTGGCCAA,0.485714,-4.8


# Recap:

The entire point of writing this function was to answer a question about Apta-MCTS. That is: "Although this program spits out a new batch of solutions everytime I run it, with no repeats, is it the case that in each batch there are solutions which are 'sampling' the area around old solutions? -- In this way, there should be a limited number of optimal solutions, and so even though the number of solutions generated by Apta-MCTS increases, the number of optimal solutions shoud stay relatively constant."

**In other words: "Will Apta-MCTS converge on a set of optimal solutions, if I run it enough times?"**

What I didn't count on was the large number of equivalent structures: Sequences which are related to eachother(in the same neighborhood), and which have the same API_score.

Could use minimum free-energy as the final selection criteria? -- That is: Ammend the 'API' function to pair down impasse-solutions (solutions in same neighborhood with the same API-score), based on free-energy change.

To try and prove the above point, what I would aim to do is generate a large number of datasets, by running Apta-MCTS atleast 10 times, with the same parameters. Then, for all possible combinations of n datasets, where n = [1, 2, ... 10], plot a graph of the number of input sequences, vs the number output sequences (crucially, all using the same parameters for the 'kill_neighbors' function). If the curve looks hyperbolic, then I have my answer.

### Anecdotally:

For a non-rigorous version of the above test, I took only one of the .csv outputs and ran 'kill_neighbors' around it.

In [ ]:
All_Sequences, API_dict, E_dict, pairwise_tn_df = make_necessary_objects(file_list=['Exp_1_1.csv','Exp_2_1.csv'])

In [ ]:
#The only one of these that the user really might need to see.....
All_Sequences

,aptamer_protein_interaction_score,primary_sequence,secondary_structure,minimum_free_energy,file
0,0.400000,AACCGGTTUAAAUAAUUUACGUUTTGGCCAA,....((((.((((.......)))).))))..,-3.4,Exp_1_1.csv
1,0.400000,AACCGGTTAAUUAAUUUACGUGGTTGGCCAA,....(((((((((.......)))))))))..,-6.9,Exp_1_1.csv
2,0.400000,AACCGGTTAAUAAUUUACGCUAATTGGCCAA,....(((((((...........)))))))..,-4.7,Exp_1_1.csv
3,0.400000,AACCGGTTCAAAUAAUUUACGUUTTGGCCAA,....(((.((((..........)))))))..,-3.6,Exp_1_1.csv
4,0.400000,AACCGGTTCUGUAAUUUACGUGGTTGGCCAA,(((((((..........)).)))))......,-2.9,Exp_1_1.csv
...,...,...,...,...,...
95,0.371429,AACCGGTTUGGUUUGGUUGCGUGTTGGCCAA,((((((......))))))((......))...,-5.3,Exp_2_1.csv
96,0.371429,AACCGGTTUAGUACUGGUUUACGTTGGCCAA,(((((((.....)))))))............,-6.9,Exp_2_1.csv
97,0.371429,AACCGGTTAUUUGAUUGGUUGAGTTGGCCAA,((((((((....))))))))...........,-5.9,Exp_2_1.csv
98,0.371429,AACCGGTTGGAAUCUGGUAGCCUTTGGCCAA,.(((((.......))))).(((...)))...,-7.4,Exp_2_1.csv


### Result:

In [ ]:
test = kill_neighbors(pairwise_tn_df, API_dict, E_dict, D=0.4, alg='API', lim=100, show='')
test

,sequence,API_score,Free_Energy_Change
0,AACCGGTTAAGUUGUGGUUUAUGTTGGCCAA,0.485714,-4.8
1,AACCGGTTAAUUGUGGUUUAUGUTTGGCCAA,0.485714,-4.7
2,AACCGGTTGUGUGGUUUAUGAGCTTGGCCAA,0.485714,-3.6
3,AACCGGTTACGUUGUGGUUUAUGTTGGCCAA,0.485714,-5.4
4,AACCGGTTGUUGUGGUUUAUGUCTTGGCCAA,0.485714,-4.1
5,AACCGGTTGCGUUGUGGUUUAUGTTGGCCAA,0.485714,-5.0
6,AACCGGTTCGUUGUGGUUUAUGUTTGGCCAA,0.485714,-3.6
7,AACCGGTTAUGUUGUGGUUUAUGTTGGCCAA,0.485714,-4.2
8,AACCGGTTAUGUGGUUUAUGUUGTTGGCCAA,0.485714,-3.2
9,AACCGGTTAGUUGUGGUUUAUGGTTGGCCAA,0.485714,-4.8


The results are quite striking. Using the file 'Exp_2_1.csv', the file contains 100 sequences, and outputs 19 of them as solutions. Recall looking at the dendrogram from 'Tamura_Nei Clustering' that many of the best solutions in each cluster were from this file. Recall also that running all 5 files, with 500 sequences, the total number of solutions rendered was 26.

Using only the file 'Exp_1_1.csv', it returns 11 solutions, all with fairly low API scores.

Finally, using both of the two files, the resulting set contains 200 sequences, but the solution set contains only 19 sequences. This implies that all of the sequences in both files were in close proximity of eachother.

This implies that all the

Finally: I notice visually, from looking at the results,that some of the solutions eg. 17 & 18, have the same "motif". In this case"GUUGUGGUUUAUGG". They have the same interaction score, but the one has greater free energy change, and so is likely to exist more stably in solution.

In [ ]:
tamura_nei('AACCGGTTGUUGUGGUUUAUGGCTTGGCCAA', 'AACCGGTTUGUUGUGGUUUAUGGTTGGCCAA')

['AACCGGTTGUUGUGGUUUAUGGCTTGGCCAA',
 'AACCGGTTUGUUGUGGUUUAUGGTTGGCCAA',
 '0.463821']

The measurement is a bit flawed, unfortunately. These are basically the same sequence, but the model thinks they could not be any more different. Not sure how to fix that.

A better example is possible 16 & 17. These two sequences have the aforementioned motif, same orderm and the same API score. One has a greater free energy change, due to a pair of substitutions.

In [ ]:
tamura_nei('AACCGGTTGUUGUGGUUUAUGGGTTGGCCAA', 'AACCGGTTGUUGUGGUUUAUGGCTTGGCCAA')

['AACCGGTTGUUGUGGUUUAUGGGTTGGCCAA',
 'AACCGGTTGUUGUGGUUUAUGGCTTGGCCAA',
 '0.0331097']

This is certainly an "impasse solution". The two are essentially the same, and are definitely "neighbors". The reason why one of them wasn't struck down is most likely that the algorithm only accounts for API score.

This a good argument for using free energy to assist in selecting solutions. Unless... perhaps it is useful in identifying where the motif is.

**Deploying the above solution, I've modified the 'API' component of the 'hit_list' function, to create a function called 'API_E'. The function should now discriminate among impasse solutions in a neighborhood via free energy change.**

## Testing API_E:

In [ ]:
test = kill_neighbors(pairwise_tn_df, API_dict, E_dict, D=0.4, alg='API_E', lim=100, show='')
test

,sequence,API_score,Free_Energy_Change
0,AACCGGTTGUUGUGGUUUAUGGCTTGGCCAA,0.485714,-6.2
1,AACCGGTTAGGUUGUGGUUUAUGTTGGCCAA,0.485714,-7.4


Idea: 'Kill-Neighbors-Dendrogram'.

It's a simple merging of the dendrogram I made in the 'Tamura-Nei Clustering' colab project, and this function that weeds out unwanted sequences. Together, I should be able to display the whole thing as a table: Show each sequence (in heatmap style, with different nucleotides color-coded), A column giving the API-score, A column giving free energy change, And finally a column giving "dot/no dot", the results of the kill-neighbors process.

Collate solutions for 1 & 16, and see if you get convergence.
Try it with a pair of unrelated AAs.

Convergence criteria. The idea is that out of all possible sequences Apta-MCTS produces, there should just one (or maybe a handful), that are the optimal solution. That give the highest API score and lowest free energy. The issue would be sampling, in that Apat-MCTS cannot check all of these prossible solutions, and so must pick a few sequences as random seeds to start from. So, the outputs of the program are likely sampling around the area of all possible solutions.

The idea then is that the more times you run the program, the more heavily you are sampling the area of all possible solutions, and the more likely you are hit the optimal solution, or close to it. So, finding the best candidate aptamer becomes an optimization problem.